### MVP Isochrome Plots - V1.1

- geocoding documentation: https://openrouteservice.org/dev/#/api-docs/geocode/search/get
- isochrome documentation: https://openrouteservice.org/dev/#/api-docs/v2/isochrones/{profile}/post

In [1]:
import geopandas as gpd 
import pandas as pd
import numpy as np
import requests
import json
import os
import sys 
import matplotlib.pyplot as plt
from openrouteservice import client
import folium
import shapely
import contextily as cx
from matplotlib_scalebar.scalebar import ScaleBar


In [77]:
#Inputs
address = '171, Newark Avenue, Peterborough'
time = 12
mode = ["driving-car", "driving-hgv", "foot-walking",
        "foot-hiking", "cycling-regular", "cycling-road", "cycling-mountain",
        "cycling-electric"]


In [78]:
#Geocoding
key = '5b3ce3597851110001cf624896366d86ef2d40439d52f50412724477'
ORS = client.Client(key=key)

location = ORS.pelias_search(text=address, size= 1, country='GBR', sources=['osm'])

#ISO
iso_car = ORS.isochrones(
        locations= [location['features'][0]['geometry']['coordinates']], 
        range = [time*60], 
        range_type='time', 
        smoothing=1, 
        attributes=['reachfactor'],
        profile= mode[0]
)

iso_walk = ORS.isochrones(
        locations= [location['features'][0]['geometry']['coordinates']], 
        range = [time*60], 
        range_type='time', 
        smoothing=1, 
        attributes=['reachfactor'],
        profile= mode[2]
)

iso_bike = ORS.isochrones(
        locations= [location['features'][0]['geometry']['coordinates']], 
        range = [time*60], 
        range_type='time', 
        smoothing=1, 
        attributes=['reachfactor'],
        profile= mode[4]
)


#params

coords = location['features'][0]['geometry']['coordinates']
coords_r = list(reversed(location['features'][0]['geometry']['coordinates']))
name = location['features'][0]['properties']['label']
transport_mode = iso_car['metadata']['query']['profile']

#json to geopandas
address_gdf =  gpd.GeoDataFrame(location['features'][0]['properties'], index=[0])

print(name)


Newark Avenue, Peterborough, England, United Kingdom


In [79]:
# Set up folium map

map1 = folium.Map(tiles='OpenStreetMap', location= coords_r, zoom_start=13 , title= 'Map',control_scale = True, attr='Nene Valley Transport Planning', width=1200, height=700)


folium.features.GeoJson(iso_car, name= iso_car['metadata']['query']['profile'],
style_function=lambda feature: {
        'fillColor': 'blue',
        'color' :'blue' }).add_to(map1) # Add GeoJson to map

folium.features.GeoJson(iso_walk, name= iso_walk['metadata']['query']['profile'],
style_function=lambda feature: {
        'fillColor': 'green',
        'color' :'green' }).add_to(map1) # Add GeoJson to map

folium.features.GeoJson(iso_bike, name= iso_bike['metadata']['query']['profile'],
style_function=lambda feature: {
        'fillColor': 'orange',
        'color' :'orange' }).add_to(map1) # Add GeoJson to map

folium.map.Marker(coords_r,  # reverse coords due to weird folium lat/lon syntax
                    icon=folium.Icon(color='lightgray',
                                    icon_color='#cc0000',
                                    icon='home',
                                    prefix='fa',
                                    ),
                    popup=name,
                    ).add_to(map1)


loc = name+' - Isochrome Catchment '+str(time)+'mins'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   


map1.get_root().html.add_child(folium.Element(title_html))

                    
folium.LayerControl().add_to(map1)

map1

In [87]:
#Points to shalpely point 
points_Car= []
for i in iso_car['features'][0]['geometry']['coordinates'][0]:
    points_Car.append(shapely.geometry.Point(i[0], i[1]))

points_walk= []
for i in iso_walk['features'][0]['geometry']['coordinates'][0]:
    points_walk.append(shapely.geometry.Point(i[0], i[1]))

points_bike= []
for i in iso_bike['features'][0]['geometry']['coordinates'][0]:
    points_bike.append(shapely.geometry.Point(i[0], i[1]))


#json to geopandas

car = address_gdf
walk = address_gdf
bike = address_gdf

car['geometry'] = shapely.geometry.Polygon([[p.x, p.y] for p in points_Car])
car = car.set_crs(4326)
car = car.to_crs(3857)

walk['geometry'] = shapely.geometry.Polygon([[p.x, p.y] for p in points_walk])
walk = walk.set_crs(4326)
walk = walk.to_crs(3857)

bike['geometry'] = shapely.geometry.Polygon([[p.x, p.y] for p in points_bike])
bike = bike.set_crs(4326)
bike = bike.to_crs(3857)

In [88]:
location_df = gpd.GeoDataFrame(location['features'][0]['properties'], index=[0])
location_df['geometry'] = shapely.geometry.Point(coords[0],coords[1])
location_df.set_crs(4326, inplace=True)
location_df.to_crs(3857, inplace=True)

In [97]:
fig, ax = plt.subplots(figsize = (13,13), facecolor='white')

car.plot(ax=ax ,alpha=0.2, edgecolor='blue', lw=4,color = 'blue' )
walk.plot(ax=ax ,alpha=0.2, edgecolor='green', lw=4, color='green' )
bike.plot(ax=ax ,alpha=0.2, edgecolor='orange', lw=4, color = 'orange')

location_df.plot(ax=ax, edgecolor='k', markersize=50, label='Car')
location_df.plot(ax=ax, edgecolor='k', markersize=50, label='Bike')
location_df.plot(ax=ax, edgecolor='k', markersize=50, label='Walk')

location_df.plot(ax=ax, color='red', edgecolor='k', markersize=70, label='Location')

cx.add_basemap(ax, url = cx.providers.OpenStreetMap.Mapnik, reset_extent=True, interpolation='sinc')


props = dict(boxstyle='round', facecolor='white', alpha=0.8)
cx.add_attribution(text='Nene Valley Transport Planning | \n© openrouteservice.org by HeiGIT | Map data © OpenStreetMap contributors ', 
    ax=ax,bbox=props)

plt.title(name+' - Isochrome Catchment '+str(time)+'mins', fontweight='bold')

ax.add_artist(
    ScaleBar(1, dimension="si-length", 
    units="m",location='lower right',
    color='k', 
    box_color='white',
    box_alpha=0.8)
 )

#Now making a nice legend
ax.legend(loc='upper left', prop={'size': 10})


#ax.set_axis_off()

plt.savefig('map.png', dpi=300)